In [ ]:
import pathlib
import datetime
import os

import dotenv
import pandas as pd
import plotly.express as px
from tqdm.auto import tqdm
from usskscadaocclient import ScadaConnector

from scrap_core.home_scrap_estimation import estimate_weekly_home_scrap_production
from scrap_core.home_scrap_estimation.common import DEFAULT_CONFIG_ENV, get_home_scrap_and_steel_production_data, \
    DEFAULT_SCRAP_TYPE_MAP_NAME

In [ ]:
dotenv.load_dotenv(DEFAULT_CONFIG_ENV)

print(os.environ.get("USE_DB"))

In [ ]:
SCADA_DB = {
    "host": os.environ.get("SCADA_DB_HOST"),
    "port": os.environ.get("SCADA_DB_PORT"),
    "database": os.environ.get("SCADA_DB_SID"),
    "username": os.environ.get("SCADA_DB_USER"),
    "password": os.environ.get("SCADA_DB_PASSWORD"),
}

scada_db = ScadaConnector(conf=SCADA_DB)

In [ ]:
workdir = pathlib.Path(r"C:\Users\dec4288\OneDrive - United States Steel\ProjectData\scrap\scrap_supplies")

dt_from = datetime.date(2023, 1, 1)
dt_to = datetime.date(2023, 8, 11)

HOME_SCRAP_GROUPS = {
    "mill revert": ["HSK", "HSR Cr", "HSR", "HS", "HSZ", "HSB", "HST", "HDS"],
    "demo": ["HSD"],
    "pit": ["BPIT", "SRB", "DSI", "1PIT", "2PIT", "PIG IRON"]
}

In [ ]:
def aggregate_home_scrap_groups(dataframe):
    dataframe = dataframe.copy()
    
    for col, group in HOME_SCRAP_GROUPS.items():
        dataframe[col] = dataframe[group].sum(axis=1)
        
    return dataframe


def find_purchase_date(dt, candidates):
    return max(c for c in candidates if c <= dt)


## Expected Home Scrap by TMS

In [ ]:
df_tms_agg = pd.read_excel(
    workdir / "odhad generacie domaceho srotu TMS 2.xlsx",
    index_col=0,
    sheet_name="home scrap + steel"
).T

df_tms_agg *= 1000

df_tms_agg

## Expected Home Scrap by Digital Studio Europe

In [ ]:
estimate = {}

for dt, record in tqdm(df_tms_agg.iterrows()):
    if pd.isna(record["steel"]):
        print(f"Missing expected steel production for {dt}")
    else:
        estimate[dt] = estimate_weekly_home_scrap_production(scada_db, dt, record["steel"] / 4)


In [ ]:
df_digital = pd.DataFrame(estimate).T * 4

df_digital_agg = aggregate_home_scrap_groups(df_digital)

df_digital_agg

## (Real) Home Scrap Production and Steel

In [ ]:
df_real = get_home_scrap_and_steel_production_data(workdir, DEFAULT_SCRAP_TYPE_MAP_NAME, scada_db, dt_from, dt_to)

df_real = df_real.rename(columns={"final_steel_w": "steel"})

df_real.head()

In [ ]:
df_real["purchase_date"] = [find_purchase_date(dt, df_tms_agg.index) for dt in df_real.index]

df_real.tail()

In [ ]:
df_real_agg = aggregate_home_scrap_groups(df_real.groupby("purchase_date").sum())

df_real_agg

## Comparison

In [ ]:
px.line(df_tms_agg.join(df_real_agg[["mill revert", "demo", "pit", "steel"]], lsuffix=" (TMS)").join(df_digital_agg[["mill revert", "demo", "pit"]], rsuffix=" (D)"))